In [1]:
library(viper)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Welcome to Bioconductor

    

## Import TM-10X single cell expressioin dataset

In [7]:
sce_droplet <- readRDS( "/gpfs/gss1/work/sduknn/Andreas/TM_MCA/TM/droplet/droplet_sce.rds")

In [9]:
sce_droplet

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: S4Vectors

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:base’:

    expand.grid

Loading required package: IRanges
Loading required package: GenomeInfoDb
Loading required package: DelayedArray
Loading required package: matrixStats

Attaching package: ‘matrixStats’

The following objects are masked from ‘package:Biobase’:

    anyMissing, rowMedians

Loading required package: BiocParallel

Attaching package: ‘DelayedArray’

The following objects are masked from ‘package:matrixStats’:

    colMaxs, colMins, colRanges, rowMaxs, rowMins, rowRanges

The following objects are masked from ‘package:base’:

    aperm, apply



class: SingleCellExperiment 
dim: 23433 70118 
metadata(0):
assays(1): counts
rownames(23433): 0610005C13Rik 0610007C21Rik ... l7Rn6
  zsGreen_transgene
rowData names(0):
colnames(70118): 10X_P4_0_AAACCTGAGATTACCC 10X_P4_0_AAACCTGAGTGCCAGA
  ... 10X_P8_15_TTTGTCATCTTACCGC 10X_P8_15_TTTGTCATCTTGTTTG
colData names(2): type tissue
reducedDimNames(0):
spikeNames(0):

## convert from SingleCellExperiment to ExpressionSet objects

In [10]:
counts <- assays(sce_droplet)$counts

In [11]:
phenoData <- colData(sce_droplet)

In [12]:
featureData <- rowData(sce_droplet)

In [13]:
library(Biobase)

# Create ExpressionSet object
eset <- ExpressionSet(assayData = counts,
                      phenoData = as(as.data.frame(phenoData), "AnnotatedDataFrame"),
                      featureData = as(as.data.frame(featureData), "AnnotatedDataFrame"))


In [14]:
eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 23433 features, 70118 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: 10X_P4_0_AAACCTGAGATTACCC 10X_P4_0_AAACCTGAGTGCCAGA ...
    10X_P8_15_TTTGTCATCTTGTTTG (70118 total)
  varLabels: type tissue
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

## Load inferred regulons 

In [52]:
regul <- aracne2regulon('VIPER_regulon.tsv', eset, verbose = TRUE, format = '3col')


Loading the dataset...
Generating the regulon objects...


number of iterations= 116 


mu: 0.103453077695501, 0, 0.261276323217341. sigma: 0.0547093213635645, 0.0349557131147495, 0.135268275311152


In [53]:
print(regul)

Object of class regulon with 222 regulators, 10119 targets and 78742 interactions


## Perform test on B cells vs rest

In [24]:
phenoData(eset)$isB <- phenoData(eset)$type == 'B cell'

In [25]:
phenoData(eset)$isB <- factor(phenoData(eset)$isB , labels = c('Other', 'B cell') ) 

In [37]:
signature <- rowTtest(eset, "isB", 'B cell', "Other")

## Run VIPER

In [39]:
signature <- (qnorm(signature$p.value/2, lower.tail = FALSE) * sign(signature$statistic))[, 1]

In [45]:
nullmodel <- ttestNull(eset, "isB", 'B cell', "Other", per = 20, repos = TRUE, verbose = FALSE)

In [63]:
mrs <- msviper(signature, regul, nullmodel, verbose = FALSE)

In [68]:
sum <- summary(mrs, mrs = 222)

In [65]:
write.table(sum, 'VIPER_result.tsv')